Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) 
с сайта superjob.ru и hh.ru. 
Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). 
Получившийся список должен содержать в себе минимум:

*Наименование вакансии
*Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
*Ссылку на саму вакансию        
*Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. 
Данная структура должна быть одинаковая для вакансий с обоих сайтов. 
Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd

In [2]:
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36'}

In [3]:
url = 'https://hh.ru/search/vacancy'

In [4]:
params = {'items_on_page': '20', 
          'area': '1',
          'page' : '0', 
          'clusters': 'true',
          'enable_snippets': 'true',
          'ored_clusters': 'true',
          'search_period': '7',
          'text': 'Python'}

In [5]:
response = requests.get(url, params=params, headers=headers).text

In [6]:
dom = bs(response,'html.parser')

In [7]:
#Определяем количество страниц с вакансиями

pages = dom.find_all('a',{'data-qa':'pager-page'})
no = []
for pages_no in pages:
    no.append(int(pages_no.getText()))
max_no = max(no)
print('Всего страниц с вакансиями    ', max_no)

Всего страниц с вакансиями     100


In [8]:
vacancy_serp = dom.find_all('div', {'class': 'vacancy-serp-item'})

In [9]:
vacancy_total = []

In [10]:
for page_no in range(0, max_no):
    for vacancy in vacancy_serp:
        
        vacancy_base = {}
    
        name_vacancy = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'}).getText()
        url_vakancy = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'}).get('href').split(sep='?')[0]


        try:
            money_vacancy = vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'}).getText().replace(u'\u202f', u'')
        except:
            money_vacancy = None


        if money_vacancy == None:
            money_min  = None
            money_max  = None
            money_currency  = None
        else:
            mv = money_vacancy.split()
            lenth = len(mv)
            if lenth == 3:
                if mv[0] == 'до':
                    money_min  = None
                    money_max  = int(mv[1])
                    money_currency  = mv[2]
                else:
                    money_min  = int(mv[1])
                    money_max  = None
                    money_currency  = mv[2]
            else:
                money_min  = int(mv[0])
                money_max  = int(mv[2])
                money_currency  = mv[3]


        vacancy_base['name'] = name_vacancy
        vacancy_base['link'] = url_vakancy
        vacancy_base['min'] = money_min
        vacancy_base['max'] = money_max
        vacancy_base['currency'] = money_currency
        vacancy_total.append(vacancy_base)
    
    print('Обработана страница    ', page_no)
    page_no += 1
    params['page'] = str(page_no)
    
    response = requests.get(url, params=params, headers=headers).text
    dom = bs(response,'html.parser')
    vacancy_serp = dom.find_all('div', {'class': 'vacancy-serp-item'})
    
df = pd.DataFrame(vacancy_total)
df.to_csv('vacancies.csv', encoding="utf-8")

print('Получился вот такой массив    >>>>>>>>   ', df.shape)
print(df.head()) 

Обработана страница     0
Обработана страница     1
Обработана страница     2
Обработана страница     3
Обработана страница     4
Обработана страница     5
Обработана страница     6
Обработана страница     7
Обработана страница     8
Обработана страница     9
Обработана страница     10
Обработана страница     11
Обработана страница     12
Обработана страница     13
Обработана страница     14
Обработана страница     15
Обработана страница     16
Обработана страница     17
Обработана страница     18
Обработана страница     19
Обработана страница     20
Обработана страница     21
Обработана страница     22
Обработана страница     23
Обработана страница     24
Обработана страница     25
Обработана страница     26
Обработана страница     27
Обработана страница     28
Обработана страница     29
Обработана страница     30
Обработана страница     31
Обработана страница     32
Обработана страница     33
Обработана страница     34
Обработана страница     35
Обработана страница     36
Обработана 

In [11]:
df


,name,link,min,max,currency
0,Data Scientist (команда Поиска),https://hh.ru/vacancy/46048820,NaN,NaN,None
1,Founding Data scientist / Machine Learning Eng...,https://hh.ru/vacancy/51448522,500000.0,NaN,руб.
2,Аналитик направления цифрового маркетинга (BI\...,https://hh.ru/vacancy/49881796,170000.0,210000.0,руб.
3,Аналитик качества данных/Data Steward,https://hh.ru/vacancy/51714877,NaN,NaN,None
4,Аналитик данных,https://hh.ru/vacancy/46241509,NaN,NaN,None
...,...,...,...,...,...
1995,Системный администратор,https://hh.ru/vacancy/50277609,120000.0,NaN,руб.
1996,DevOps specialist,https://hh.ru/vacancy/51625745,NaN,NaN,None
1997,Frontend Developer,https://hh.ru/vacancy/50974753,NaN,220000.0,руб.
1998,Операционный аналитик,https://hh.ru/vacancy/46049324,NaN,NaN,None
